In [ ]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset_advanced = load_dataset('advanced_stats.csv')
dataset_allgames = load_dataset('allgames_stats.csv')
dataset_allstar = load_dataset('allstar_games_stats.csv')
dataset_gamehigh = load_dataset('game_highs_stats.csv')
dataset_pergame = load_dataset('per_game_stats.csv')
dataset_salaries = load_dataset('salaries.csv')
dataset_totals = load_dataset('totals_stats.csv')

data_advanced = dataset_advanced.values
data_allgames = dataset_allgames.values
data_allstar = dataset_allstar.values
data_gamehigh = dataset_gamehigh.values
data_pergame = dataset_pergame.values
data_salaries = dataset_salaries.values
data_totals = dataset_totals.values

print("Dimensionalitat de la BBDD:", dataset_advanced.shape)
print("Dimensionalitat de la BBDD:", dataset_allgames.shape)
print("Dimensionalitat de la BBDD:", dataset_allstar.shape)
print("Dimensionalitat de la BBDD:", dataset_gamehigh.shape)
print("Dimensionalitat de la BBDD:", dataset_pergame.shape)
print("Dimensionalitat de la BBDD:", dataset_salaries.shape)
print("Dimensionalitat de la BBDD:", dataset_totals.shape)

Visualització de les dades

In [ ]:
dataset_allgames.head()

In [ ]:
dataset_allgames.info()

In [ ]:
dataset_allstar.head()

In [ ]:
dataset_allstar.info()

In [ ]:
dataset_gamehigh.head()

In [ ]:
dataset_gamehigh.info()

In [ ]:
dataset_pergame.head()

In [ ]:
dataset_pergame.info()

In [ ]:
dataset_salaries.head()

In [ ]:
dataset_salaries.info()

In [ ]:
dataset_totals.head()

In [ ]:
dataset_totals.info()

In [ ]:
# dataset_allgames.info()
# dataset_copy2 = dataset_allgames.copy()
# lebron = []
# jordan = []
# kobe = []
# for i in dataset_copy2["Player"]:
#     if i == 'Lebron James':
#         lebron.append(1)
#         jordan.append(0)
#         kobe.append(0)
#     if i == 'Michael Jordan':
#         lebron.append(0)
#         jordan.append(1)
#         kobe.append(0)
#     if i == 'Kobe Bryant':
#         lebron.append(0)
#         jordan.append(0)
#         kobe.append(1)
# victories = []
# for i in dataset_copy2["Result"]:
#     if i == 'W':
#         victories.append(1)
#     if i == 'L':
#         victories.append(0)
# dataset_copy2 = dataset_copy2.assign(Lebron = lebron)
# dataset_copy2 = dataset_copy2.assign(Jordan = jordan)
# dataset_copy2 = dataset_copy2.assign(Kobe = kobe)
# dataset_copy2 = dataset_copy2.assign(Victories = victories)
# print(dataset_copy2.isnull().sum())

Estudi del dataset amb el que es treballarà.

In [ ]:
dataset_advanced.head()

In [ ]:
dataset_advanced.info()

In [ ]:
dataset_copy = dataset_advanced.copy()

Creació de 3 columnes noves per indicar de quin jugador es tracta

In [ ]:
lebron = []
jordan = []
kobe = []
for i in dataset_copy["Player"]:
    if i == 'Lebron James':
        lebron.append(1)
        jordan.append(0)
        kobe.append(0)
    if i == 'Michael Jordan':
        lebron.append(0)
        jordan.append(1)
        kobe.append(0)
    if i == 'Kobe Bryant':
        lebron.append(0)
        jordan.append(0)
        kobe.append(1)

dataset_copy = dataset_copy.assign(Lebron = lebron)
dataset_copy = dataset_copy.assign(Jordan = jordan)
dataset_copy = dataset_copy.assign(Kobe = kobe)
dataset_copy.info()
dataset_copy.head()

In [ ]:
dataset_copy.drop(['Player'], axis = 1)
correlacio = dataset_copy.corr()
correlacio["PER"]

Primers estudis de les correlacions per veure els atributs més rellevants

In [ ]:
l = []
objectiu = "PER"
for i in range(len(correlacio[objectiu])):
    if (correlacio[objectiu][i]) > 0.5 and correlacio[objectiu][i] != 1:
        l.append(i)
print(correlacio[objectiu][l])
print(l)

In [ ]:
names_corr = ['PER','TS%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'USG%', 'OWS', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [ ]:
import seaborn as sns
dataset_cor = dataset_copy.copy()

for i in dataset_copy:
    if i not in names_corr:
        dataset_cor = dataset_cor.drop(i, axis = 1)

correlacio = dataset_cor.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)

In [ ]:
import seaborn as sns
#relacio = sns.pairplot(dataset_cor)

In [ ]:
dataset_cor.head()

Estandarització de les dades e histogrames per observar si segueixen algun tipus de distribució útil per l'estudi de la regressió.

In [ ]:
def standarize(X): 
    mean = X.mean(0)
    std = X.std(0)
    x = X - mean
    x /= std
    return x

for i in dataset_cor:
    X = standarize(dataset_cor[i].values)
    plt.figure()
    plt.title("Histograma de l'atribut "+str(i))
    plt.xlabel("Attribute Value")
    plt.ylabel("Count")
    hist = plt.hist(X, bins=11, range=[np.min(X), np.max(X)], histtype="bar", rwidth=0.8)
    plt.plot()

Separació del dataset original en 3 de diferents per tal de predir la variable objectiu PER per a cada jugador. D'aquesta manera, el jugador que obtingui un valor més elevat en la predicció serà el millor.

In [ ]:
names_aux = ['PER','TS%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'USG%', 'OWS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Lebron', 'Jordan','Kobe']
dataset_aux = dataset_copy.copy()

for i in dataset_copy:
    if i not in names_aux:
        dataset_aux = dataset_aux.drop(i, axis = 1)

data_Lebron = dataset_aux.loc[dataset_aux["Lebron"]==1]
data_Lebron.head()

In [ ]:
data_Jordan = dataset_aux.loc[dataset_aux["Jordan"]==1]
data_Jordan.head()

In [ ]:
data_Kobe = dataset_aux.loc[dataset_aux["Kobe"]==1]
data_Kobe.head()

Estudi dels atributs de Lebron

In [ ]:
for i in data_Lebron:
    if i not in ["Lebron", "Jordan", "Kobe"]:
        X = standarize(data_Lebron[i].values)
        plt.figure()
        plt.title("Histograma de l'atribut "+str(i))
        plt.xlabel("Attribute Value")
        plt.ylabel("Count")
        hist = plt.hist(X, bins=11, range=[np.min(X), np.max(X)], histtype="bar", rwidth=0.8)
        plt.plot()

Estudi dels atributs de Jordan

In [ ]:
for i in data_Jordan:
    if i not in ["Lebron", "Jordan", "Kobe"]:
        X = standarize(data_Jordan[i].values)
        plt.figure()
        plt.title("Histograma de l'atribut "+str(i))
        plt.xlabel("Attribute Value")
        plt.ylabel("Count")
        hist = plt.hist(X, bins=11, range=[np.min(X), np.max(X)], histtype="bar", rwidth=0.8)
        plt.plot()

Estudi dels atributs de Kobe

In [ ]:
for i in data_Kobe:
    if i not in ["Lebron", "Jordan", "Kobe"]:
        X = standarize(data_Kobe[i].values)
        plt.figure()
        plt.title("Histograma de l'atribut "+str(i))
        plt.xlabel("Attribute Value")
        plt.ylabel("Count")
        hist = plt.hist(X, bins=11, range=[np.min(X), np.max(X)], histtype="bar", rwidth=0.8)
        plt.plot()

Pairplot per l'estudi de les distribucions i relacions entre atributs més rellevants per cada jugador

In [ ]:
dl = data_Lebron.copy()

In [ ]:
dl = data_Lebron.copy()

for i in data_Lebron:
    if i not in names_corr:
        dl = dl.drop(i, axis = 1)

correlacio = dl.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dl)

In [ ]:
dj = data_Jordan.copy()

In [ ]:


for i in data_Jordan:
    if i not in names_corr:
        dj = dj.drop(i, axis = 1)

correlacio = dj.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dj)

In [ ]:
dk = data_Kobe.copy()

In [ ]:


for i in data_Kobe:
    if i not in names_corr:
        dk = dk.drop(i, axis = 1)

correlacio = dk.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dk)

Definició de funcions usades en l'estudi del dataset i les prediccions.

In [ ]:
import warnings

In [ ]:
import math

def mse(y1, y2, inici):
    # comprovem que y1 i y2 tenen la mateixa mida
    assert(len(y1) == len(y2))
    mse = 0
    i = []
    j = []
    for aux in range(inici, inici+len(y1)):
        i.append(aux)
    for aux in range(0, len(y1)):
        j.append(aux)
    for index1, index2 in zip(i,j):
        mse += (y1[index1] - y2[index2])**2
    return mse / len(y1)

In [ ]:
from sklearn.linear_model import LinearRegression

def regression(x, y):
    # Creem un objecte de regressió de sklearn
    regr = LinearRegression()
    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)
    # Retornem el model entrenat
    return regr

In [ ]:
def split_data(x, y, train_ratio=0.8):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val

In [ ]:
from sklearn import linear_model

def lasso(x, y, a = 0.1):

    # Creem un objecte de regressió de sklearn
    regr = linear_model.Lasso(alpha=a)

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

In [ ]:
from sklearn.linear_model import BayesianRidge

def Bayes(x, y, t = 1e-6):

    # Creem un objecte de regressió de sklearn
    regr = BayesianRidge(tol= t)

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

In [ ]:
from sklearn.linear_model import ElasticNet

def ElasNet(x, y, t = 1e-6):

    # Creem un objecte de regressió de sklearn
    regr = ElasticNet(random_state=0)

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

Es decideix fer, per cada jugador:
    - Regressor Lineal (estandaritzat i no estandaritzat)
    - Regressor Multilineal amb els 3 atributs comuns als 3 jugadors
    - Regressor Multilineal amb búsqueda de la combinació d'atributs més rellevants repetint l'experiment 100 cops per tenir uns valors "estables"
    - PCA
    - Lasso
    - BayesianRidge
    - Polinomial
    - ElasticNet
    - Aplicació del CrossValidation

Procedim a estudiar quin es el millor regressor possible per a predir el "PER" mitjançant les dades i el resultats obtinguts en el anàlisis. Fem la recta de predicció del "PER" amb els atributs més rellevants mencionats abans i estudiant la qualitat de la seva predicció.

In [ ]:
pip install plotly

In [ ]:
from sklearn.metrics import r2_score
data_Lebron_no_estandaritzat = dataset_aux.loc[dataset_aux["Lebron"]==1]

Búsqueda dels atributs més rellevants per a cada jugador que s'usaran en les prediccions

In [ ]:
l = []
j = []
k = []
s = "PER"

correlacio_Lebron = dl.corr()
correlacio_Jordan = dj.corr()
correlacio_Kobe = dk.corr()

for i in range(len(correlacio_Lebron[s])):
    if abs(correlacio_Lebron[s][i]) > 0.6 and correlacio_Lebron[s][i] != 1:
        l.append(i)
print(correlacio_Lebron[s][l])
print("-----------------------")
for i in range(len(correlacio_Jordan[s])):
    if abs(correlacio_Jordan[s][i]) > 0.6 and correlacio_Jordan[s][i] != 1:
        j.append(i)
print(correlacio_Jordan[s][j])
print("-----------------------")
for i in range(len(correlacio_Kobe[s])):
    if abs(correlacio_Kobe[s][i]) > 0.6 and correlacio_Kobe[s][i] != 1:
        k.append(i)
print(correlacio_Kobe[s][k])

Observem que els atributs comuns són TS%, OBPM i BPM

Estudi correlacions per cada jugador

In [ ]:
#Pairplot Lebron
names_corr = ["TS%", "DRB%", "TRB%", "OBPM", "BPM", "PER"]

dl_pairplot = data_Lebron.copy()

for i in data_Lebron:
    if i not in names_corr:
        dl_pairplot = dl_pairplot.drop(i, axis = 1)

correlacio = dl_pairplot.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dl_pairplot)

In [ ]:
data_Lebron_cp = data_Lebron.copy()

y = np.array(data_Lebron['PER'])
for i in data_Lebron_cp:
    if i not in names_corr:
        data_Lebron_cp = data_Lebron_cp.drop(i, axis = 1)

x = data_Lebron_cp.to_numpy()

In [ ]:
#REPRESENTACIÓ 2 A 2 EN R3
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Lebron_cp.to_numpy()
M = M.transpose()

l = [i for i in range(np.shape(M)[0])]
n = names_corr

z = np.array(data_Lebron['PER'])


for i in l:
    for j in l:
        if j > i:
            print("PER: ",n[i],"--", n[j])
            # Creamos la figura
            fig = plt.figure()
            # Creamos el plano 3D
            ax = fig.gca(projection='3d')

            # Definimos los datos de prueba
            x = M[i]
            y = M[j]

            # Agregamos los puntos en el plano 3D
            ax.scatter(x, y, z, c='g', marker='o')

            # Mostramos el gráfico
            plt.show()

In [ ]:
#REPRESENTACIÓ 3 A 3 EN R3
# Scatter
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Lebron_cp.to_numpy()
M = M.transpose()

l = [[0,1,2],[1,2,3],[2,3,4],[3,4,0], [4,0,1]]
n = names_corr

a = np.array(data_Lebron['PER'])
a = a/max(a)
v = [i for i in range(len(a))]

for i in l:
    print("PER: ",n[i[0]],"--", n[i[1]], "--", n[i[2]])
    # Creamos la figura
    fig = plt.figure()
    # Creamos el plano 3D
    ax = fig.gca(projection='3d')

    # Definimos los datos de prueba
    x = M[i[0]]
    y = M[i[1]]
    z = M[i[2]]

    # Agregamos los puntos en el plano 3D
    img = ax.scatter(x, y, z, c = np.take(a, v), marker='o')
    
#     surface = ax.plot_surface(x,y,z, cmap = "summer")
    fig.colorbar(img)

    # Mostramos el gráfico
    plt.show()

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA


features = names_corr

pca = PCA()
components = pca.fit_transform(data_Lebron[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=data_Lebron["PER"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
# Pairplot Jordan
dj_pairplot = data_Jordan.copy()
names_corr=["TS%", "STL%", "OBPM", "BPM", "PER"]
for i in data_Jordan:
    if i not in names_corr:
        dj_pairplot = dj_pairplot.drop(i, axis = 1)

correlacio = dj_pairplot.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dj_pairplot)

In [ ]:
data_Jordan_cp = data_Jordan.copy()

y = np.array(data_Jordan['PER'])
for i in data_Jordan_cp:
    if i not in names_corr:
        data_Jordan_cp = data_Jordan_cp.drop(i, axis = 1)

x = data_Jordan_cp.to_numpy()

In [ ]:
#REPRESENTACIÓ 2 A 2 EN R3
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Jordan_cp.to_numpy()
M = M.transpose()

l = [i for i in range(np.shape(M)[0])]
n = names_corr

z = np.array(data_Jordan['PER'])


for i in l:
    for j in l:
        if j > i:
            print("PER: ",n[i],"--", n[j])
            # Creamos la figura
            fig = plt.figure()
            # Creamos el plano 3D
            ax = fig.gca(projection='3d')

            # Definimos los datos de prueba
            x = M[i]
            y = M[j]

            # Agregamos los puntos en el plano 3D
            ax.scatter(x, y, z, c='g', marker='o')

            # Mostramos el gráfico
            plt.show()

In [ ]:
#REPRESENTACIÓ 3 A 3 EN R3
# Scatter
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Jordan_cp.to_numpy()
M = M.transpose()

l = [[0,1,2],[1,2,3],[2,3,4],[3,4,0], [4,0,1]]
n = names_corr

a = np.array(data_Jordan['PER'])
a = a/max(a)
v = [i for i in range(len(a))]

for i in l:
    print("PER: ",n[i[0]],"--", n[i[1]], "--", n[i[2]])
    # Creamos la figura
    fig = plt.figure()
    # Creamos el plano 3D
    ax = fig.gca(projection='3d')

    # Definimos los datos de prueba
    x = M[i[0]]
    y = M[i[1]]
    z = M[i[2]]

    # Agregamos los puntos en el plano 3D
    img = ax.scatter(x, y, z, c = np.take(a, v), marker='o')
    
#     surface = ax.plot_surface(x,y,z, cmap = "summer")
    fig.colorbar(img)

    # Mostramos el gráfico
    plt.show()

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA


features = names_corr

pca = PCA()
components = pca.fit_transform(data_Jordan[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=data_Jordan["PER"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
#Pairplot Kobe
dk_pairplot = data_Kobe.copy()
names_corr=["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP", "PER"]
for i in data_Kobe:
    if i not in names_corr:
        dk_pairplot = dk_pairplot.drop(i, axis = 1)

correlacio = dk_pairplot.corr() 

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)
relacio = sns.pairplot(dk_pairplot)

In [ ]:
data_Kobe_cp = data_Kobe.copy()

y = np.array(data_Kobe['PER'])
for i in data_Kobe_cp:
    if i not in names_corr:
        data_Kobe_cp = data_Kobe_cp.drop(i, axis = 1)

x = data_Kobe_cp.to_numpy()

In [ ]:
#REPRESENTACIÓ 2 A 2 EN R3
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Kobe_cp.to_numpy()
M = M.transpose()

l = [i for i in range(np.shape(M)[0])]
n = names_corr

z = np.array(data_Kobe['PER'])


for i in l:
    for j in l:
        if j > i:
            print("PER: ",n[i],"--", n[j])
            # Creamos la figura
            fig = plt.figure()
            # Creamos el plano 3D
            ax = fig.gca(projection='3d')

            # Definimos los datos de prueba
            x = M[i]
            y = M[j]

            # Agregamos los puntos en el plano 3D
            ax.scatter(x, y, z, c='g', marker='o')

            # Mostramos el gráfico
            plt.show()

In [ ]:
#REPRESENTACIÓ 3 A 3 EN R3
# Scatter
%matplotlib notebook
# importamos las librerias necesarias
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact

M = data_Jordan_cp.to_numpy()
M = M.transpose()

l = [[0,1,2],[1,2,3],[2,3,4],[3,4,0], [4,0,1]]
n = names_corr

a = np.array(data_Jordan['PER'])
a = a/max(a)
v = [i for i in range(len(a))]

for i in l:
    print("PER: ",n[i[0]],"--", n[i[1]], "--", n[i[2]])
    # Creamos la figura
    fig = plt.figure()
    # Creamos el plano 3D
    ax = fig.gca(projection='3d')

    # Definimos los datos de prueba
    x = M[i[0]]
    y = M[i[1]]
    z = M[i[2]]

    # Agregamos los puntos en el plano 3D
    img = ax.scatter(x, y, z, c = np.take(a, v), marker='o')
    
#     surface = ax.plot_surface(x,y,z, cmap = "summer")
    fig.colorbar(img)

    # Mostramos el gráfico
    plt.show()

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA


features = names_corr

pca = PCA()
components = pca.fit_transform(data_Kobe[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=data_Kobe["PER"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
names_cor = ["TS%", "OBPM", "BPM"]

Regressió lineal per Lebron amb les dades no estandaritzades

In [ ]:
#REGRESSIÓ LINEAL PER LEBRON
y = data_Lebron_no_estandaritzat["PER"]
dic = {}
for i in ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]:
    if i not in ["PER", "Lebron", "Jordan", "Kobe"]:
        x = np.array(data_Lebron_no_estandaritzat[i])
        X = x.reshape(x.shape[0], 1) 
        regr = regression(X, y) 
        predicted = regr.predict(X)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        plt.figure()
        plt.title(i)
        ax = plt.scatter(x, y)
        plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 0)
        r2 = r2_score(y, predicted)
        
        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        
        print(i)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        dic[i] = (media, MSE, r2, i)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió lineal per Lebron amb les dades estandaritzades

In [ ]:
#PER LEBRON
y = np.array(dl["PER"])
dic = {}
for j in ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]:
    x = np.array(dl[j])
    x = x.reshape(x.shape[0], 1) 

    x_train, y_train, x_val, y_val = split_data(x, y)

    x_t = x_train # seleccionem atribut i en conjunt de train
    x_v = x_val # seleccionem atribut i en conjunt de val.
    x_t = np.reshape(x_t,(x_t.shape[0],1))
    x_v = np.reshape(x_v,(x_v.shape[0],1))

    regr = regression(x_t, y_train)    
    error = mse(y_val, regr.predict(x_v),0) # calculem error
    r2 = r2_score(y_val, regr.predict(x_v))
    predicted = regr.predict(x_v)
    media = 0
    for prediction in predicted:
        media += prediction
    media /= len(predicted)

    print(j)
    print(media)
    print("Error en atribut %s: %f" %(j, error))
    print("R2 score en atribut %s: %f" %(j, r2))
    print("----------------------------------------------------------------")

    dic[j] = (media, error, r2, j)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])
        

Regressió multilineal amb els 3 atributs comuns a tots els jugadors: TS%, OBPM i BPM.

In [ ]:
#LEBRON
Dl_copy = data_Lebron_no_estandaritzat.copy()

y = np.array(dl['PER'])
for i in Dl_copy:
    if i not in names_cor:
        Dl_copy = Dl_copy.drop(i, axis = 1)

x = Dl_copy.to_numpy()

x_train, y_train, x_val, y_val = split_data(x, y)

x_t = x_train # seleccionem atribut i en conjunt de train
x_v = x_val # seleccionem atribut i en conjunt de val.
# x_t = np.reshape(x_t,(x_t.shape[0],20))
# x_v = np.reshape(x_v,(x_v.shape[0],20))

regr = regression(x_t, y_train)    
error = mse(y_val, regr.predict(x_v),0) # calculem error
r2 = r2_score(y_val, regr.predict(x_v))
predicted = regr.predict(x_v)
media = 0
for prediction in predicted:
    media += prediction
media /= len(predicted)

print(media)
print("Mean squeared error: ", error)
print("R2 score: ", r2)

Regressió multilineal Lebron: búsqueda de la millor combinació d'atributs

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ MULTILINEAL LEBRON

#fem les combinacions
names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = regression(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    print(names_cor)
    print(media)
    print("Mean squeared error: ", error)
    print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió amb Lasso i búsqueda de la millor combinació d'atributs (Lebron)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO LEBRON

#fem les combinacions
names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = lasso(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió amb Bayes i búsqueda de la millor combinació d'atributs (Lebron)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ BAYES LEBRON

#fem les combinacions
names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = Bayes(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió amb ElasNet i búsqueda de la millor combinació d'atributs (Lebron)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ BAYES LEBRON

#fem les combinacions
names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = ElasNet(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

Regressió polinomial amb búsqueda del millor atribut i millor grau (Lebron)

In [ ]:
Dl_copy = data_Lebron_no_estandaritzat.copy()
y = Dl_copy["PER"]

dic = {}
for i in ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]:
    x = np.array(Dl_copy[i])
    X = x.reshape(x.shape[0], 1) 

    for grau in range (2,5):
        poly = PolynomialFeatures(degree=grau, include_bias=False)
        poly_features = poly.fit_transform(x.reshape(-1, 1))
        poly_reg_model = LinearRegression()
        poly_reg_model.fit(poly_features, y)
        predicted = poly_reg_model.predict(poly_features)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        #plt.figure()
        #plt.title(i)
        #ax = plt.scatter(x, y)
        #plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 0)
        r2 = r2_score(y, predicted)

        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)

        print(i, " amb ", grau)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        
        dic[i, " grau: ", grau] = (media, MSE, r2, i, grau)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Crossvalidation Lineal (Lebron)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
dic = {}
for names_cor in names_corr_importants:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(Dl_copy['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Crossvalidation Lasso (Lebron)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val

dic = {}
names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)
    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)
   
    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Crossvalidation Bayes (Lebron)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    media = 0
    for pred in predicted:
        media += pred
    media /= len(predicted)
    # Mostrem l'error (MSE i R2)
    print(media)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Bayes amb els millors paràmetres

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    regr = BayesianRidge(n_iter = 275, tol = 0.02, alpha_1 = 0.024, alpha_2 = 0.008, lambda_1 = 0.05, lambda_2 = 0.09)

    # Entrenem el model per a predir y a partir de x
    regr.fit(X, y)
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Crossvalidation Multilineal (Lebron)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Crossvalidation ElasNet (Lebron)

In [ ]:
warnings.filterwarnings("ignore")
from itertools import combinations
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "DRB%", "TRB%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dl_copy = data_Lebron_no_estandaritzat.copy()
    y = np.array(data_Lebron_no_estandaritzat['PER'])
    for i in Dl_copy:
        if i not in names_cor:
            Dl_copy = Dl_copy.drop(i, axis = 1)

    x = Dl_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    regr = ElasNet(x_t,y_t)
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Lineal sense estandaritzar (Jordan)

In [ ]:
#LINEAL SENSE ESTANDARITZAR PER VEURE QUIN VA MILLOR!
from sklearn.metrics import r2_score
data_Jordan_no_estandaritzat = dataset_aux.loc[dataset_aux["Jordan"]==1]
y = data_Jordan_no_estandaritzat["PER"]
dic = {}
for i in ["TS%", "STL%", "OBPM", "BPM"]:
    if i not in ["PER", "Lebron", "Jordan", "Kobe"]:
        x = np.array(data_Jordan_no_estandaritzat[i])
        X = x.reshape(x.shape[0], 1) 
        regr = regression(X, y) 
        predicted = regr.predict(X)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        plt.figure()
        plt.title(i)
        ax = plt.scatter(x, y)
        plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 29)
        r2 = r2_score(y, predicted)
        
        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        
        print(i)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        dic[i] = (media, MSE, r2, i)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])
        

Lineal estandaritzat (Jordan)

In [ ]:
y = np.array(dj["PER"])
dic = {}
for j in ["TS%", "STL%", "OBPM", "BPM"]:
    if j not in ["PER", "Lebron", "Jordan", "Kobe"]:
        x = np.array(dj[j])
        x = x.reshape(x.shape[0], 1) 

        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        x_t = np.reshape(x_t,(x_t.shape[0],1))
        x_v = np.reshape(x_v,(x_v.shape[0],1))

        regr = regression(x_t, y_train)
        error = mse(y_val, regr.predict(x_v),0) # calculem error
        r2 = r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        
        print(j)
        print(media)
        print("Error en atribut %s: %f" %(j, error))
        print("R2 score en atribut %s: %f" %(j, r2))
        print("----------------------------------------------------------------")
        dic[j] = (media, MSE, r2, j)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Multilineal amb els 3 atributs comuns: TS%, OBPM i BPM (Jordan)

In [ ]:
#JORDAN
Dj_copy = data_Jordan_no_estandaritzat.copy()

y = np.array(data_Jordan_no_estandaritzat['PER'])
for i in Dj_copy:
    if i not in names_cor:
        Dj_copy = Dj_copy.drop(i, axis = 1)

x = Dj_copy.to_numpy()

x_train, y_train, x_val, y_val = split_data(x, y)

x_t = x_train # seleccionem atribut i en conjunt de train
x_v = x_val # seleccionem atribut i en conjunt de val.
# x_t = np.reshape(x_t,(x_t.shape[0],20))
# x_v = np.reshape(x_v,(x_v.shape[0],20))

regr = regression(x_t, y_train)    
error = mse(y_val, regr.predict(x_v),0) # calculem error
r2 = r2_score(y_val, regr.predict(x_v))
predicted = regr.predict(x_v)
media = 0
for prediction in predicted:
    media += prediction
media /= len(predicted)

print(media)
print("Mean squeared error: ", error)
print("R2 score: ", r2)

Millor combinació multilinial dels atributs més rellevants (Jordan)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ MULTILINEAL JORDAN

#fem les combinacions
names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,5):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DJ_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in DJ_copy:
        if i not in names_cor:
            DJ_copy = DJ_copy.drop(i, axis = 1)

    x = DJ_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = regression(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Millor combinació Lasso (Jordan)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO JORDAN

#fem les combinacions
names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,5):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DJ_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in DJ_copy:
        if i not in names_cor:
            DJ_copy = DJ_copy.drop(i, axis = 1)

    x = DJ_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = lasso(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Millor combinació Bayes (Jordan)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO JORDAN

#fem les combinacions
names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,5):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DJ_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in DJ_copy:
        if i not in names_cor:
            DJ_copy = DJ_copy.drop(i, axis = 1)

    x = DJ_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = Bayes(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Millor combinació ElasNet (Jordan)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO JORDAN

#fem les combinacions
names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,5):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DJ_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in DJ_copy:
        if i not in names_cor:
            DJ_copy = DJ_copy.drop(i, axis = 1)

    x = DJ_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = ElasNet(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Millor atribut i grau per polinomial (Jordan)

In [ ]:
Dj_copy = data_Jordan_no_estandaritzat.copy()
y = Dj_copy["PER"]
dic = {}

for i in ["TS%", "STL%", "OBPM", "BPM"]: 
    x = np.array(Dj_copy[i])
    X = x.reshape(x.shape[0], 1) 

    for grau in range (2,5):
        poly = PolynomialFeatures(degree=grau, include_bias=False)
        poly_features = poly.fit_transform(x.reshape(-1, 1))
        poly_reg_model = LinearRegression()
        poly_reg_model.fit(poly_features, y)
        predicted = poly_reg_model.predict(poly_features)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        #plt.figure()
        #plt.title(i)
        #ax = plt.scatter(x, y)
        #plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 29)
        r2 = r2_score(y, predicted)

        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)

        print(i, " amb ", grau)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        dic[i, " grau: ", grau] = (media, MSE, r2, i, grau)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

CrossValidation regressió lineal (Jordan)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
dic = {}
for names_cor in names_corr_importants:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(Dj_copy['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation Lasso (Jordan)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(Dj_copy['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    media = 0
    for pred in predicted:
        media += pred
    media /= len(predicted)
    # Mostrem l'error (MSE i R2)
    print(media)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation amb els millors parametres pel Lasso

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(Dj_copy['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    regr = linear_model.Lasso(alpha=1.81, selection="cyclic", tol=0.00017)
    regr.fit(x_t, y_t) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation Bayes (Jordan)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation multilineal (Jordan)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation ElasNet (Jordan)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "STL%", "OBPM", "BPM"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dj_copy = data_Jordan_no_estandaritzat.copy()
    y = np.array(data_Jordan_no_estandaritzat['PER'])
    for i in Dj_copy:
        if i not in names_cor:
            Dj_copy = Dj_copy.drop(i, axis = 1)

    x = Dj_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    regr = ElasNet(X,y)
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Regressió lineal sense estandaritzar (Kobe)

In [ ]:
#LINEAL SENSE ESTANDARITZAR PER VEURE QUIN VA MILLOR!
from sklearn.metrics import r2_score
dic = {}
data_Kobe_no_estandaritzat = dataset_aux.loc[dataset_aux["Kobe"]==1]
y = data_Kobe_no_estandaritzat["PER"]
for i in ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]:
    if i not in ["PER", "Lebron", "Jordan", "Kobe"]:
        x = np.array(data_Kobe_no_estandaritzat[i])
        X = x.reshape(x.shape[0], 1) 
        regr = regression(X, y) 
        predicted = regr.predict(X)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        plt.figure()
        plt.title(i)
        ax = plt.scatter(x, y)
        plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 57)
        r2 = r2_score(y, predicted)
        
        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        
        print(i)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        dic[i] = (media, MSE, r2, i)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió lineal estandaritzat (Kobe)

In [ ]:
#PER Kobe
y = np.array(dk["PER"])
dic = {}
for j in ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]:
    if j not in ["PER", "Lebron", "Jordan", "Kobe"]:
        x = np.array(dk[j])
        x = x.reshape(x.shape[0], 1) 

        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        x_t = np.reshape(x_t,(x_t.shape[0],1))
        x_v = np.reshape(x_v,(x_v.shape[0],1))

        regr = regression(x_t, y_train)    
        error = mse(y_val, regr.predict(x_v),0) # calculem error
        r2 = r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        
        print(j)
        print(media)
        print("Error en atribut %s: %f" %(j, error))
        print("R2 score en atribut %s: %f" %(j, r2))
        print("----------------------------------------------------------------")
        dic[j] = (media, MSE, r2, j)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]

print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió multilineal amb els 3 atributs comuns TS%, OBPM i BPM (Kobe)

In [ ]:
#KOBE
Dk_copy = data_Kobe_no_estandaritzat.copy()

y = np.array(data_Kobe_no_estandaritzat['PER'])
for i in Dk_copy:
    if i not in names_cor:
        Dk_copy = Dk_copy.drop(i, axis = 1)

x = Dk_copy.to_numpy()

x_train, y_train, x_val, y_val = split_data(x, y)

x_t = x_train # seleccionem atribut i en conjunt de train
x_v = x_val # seleccionem atribut i en conjunt de val.
# x_t = np.reshape(x_t,(x_t.shape[0],20))
# x_v = np.reshape(x_v,(x_v.shape[0],20))

regr = regression(x_t, y_train)    
error = mse(y_val, regr.predict(x_v),0) # calculem error
r2 = r2_score(y_val, regr.predict(x_v))
predicted = regr.predict(x_v)
media = 0
for prediction in predicted:
    media += prediction
media /= len(predicted)

print(media)
print("Mean squeared error: ", error)
print("R2 score: ", r2)

Regressió multilinial (Kobe)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ MULTILINEAL KOBE

#fem les combinacions
names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,7):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DK_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in DK_copy:
        if i not in names_cor:
            DK_copy = DK_copy.drop(i, axis = 1)

    x = DK_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = regression(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió Lasso (Kobe)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO KOBE

#fem les combinacions
names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,7):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DK_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in DK_copy:
        if i not in names_cor:
            DK_copy = DK_copy.drop(i, axis = 1)

    x = DK_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = lasso(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió Bayes (Kobe)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO KOBE

#fem les combinacions
names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,7):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DK_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in DK_copy:
        if i not in names_cor:
            DK_copy = DK_copy.drop(i, axis = 1)

    x = DK_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = Bayes(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió ElasNet (Kobe)

In [ ]:
from itertools import combinations

#BÚSQUEDA MILLOR COMBINACIÓ LASSO KOBE

#fem les combinacions
names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,7):
    names_combinations+=combinations(names_corr_importants,i)

#for names_cor in names_combinations:
#    print(names_cor)

#Calculem les prediccions i mirem els errors mentre creem un diccionari per cercar la combinació amb menor mse i major r2
dic = {}
rep = 100
total_mitja = 0
error = 0
r2 = 0

for names_cor in names_combinations:
    DK_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in DK_copy:
        if i not in names_cor:
            DK_copy = DK_copy.drop(i, axis = 1)

    x = DK_copy.to_numpy()

    for k in range(rep):
        x_train, y_train, x_val, y_val = split_data(x, y)

        x_t = x_train # seleccionem atribut i en conjunt de train
        x_v = x_val # seleccionem atribut i en conjunt de val.
        # x_t = np.reshape(x_t,(x_t.shape[0],20))
        # x_v = np.reshape(x_v,(x_v.shape[0],20))

        regr = ElasNet(x_t, y_train)    
        error += mse(y_val, regr.predict(x_v),0) # calculem error
        r2 += r2_score(y_val, regr.predict(x_v))
        predicted = regr.predict(x_v)
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)
        total_mitja += media
    
    error /= rep
    r2 /= rep
    total_mitja /= rep
    dic[names_cor] = (total_mitja, error, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

Regressió polinomial (Kobe)

In [ ]:
Dk_copy = data_Kobe_no_estandaritzat.copy()
y = Dk_copy["PER"]
dic = {}

for i in ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]: 
    x = np.array(Dk_copy[i])
    X = x.reshape(x.shape[0], 1) 

    for grau in range (2,5):
        poly = PolynomialFeatures(degree=grau, include_bias=False)
        poly_features = poly.fit_transform(x.reshape(-1, 1))
        poly_reg_model = LinearRegression()
        poly_reg_model.fit(poly_features, y)
        predicted = poly_reg_model.predict(poly_features)

        # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
        #plt.figure()
        #plt.title(i)
        #ax = plt.scatter(x, y)
        #plt.plot(X, predicted, 'r')

        # Mostrem l'error (MSE i R2)
        MSE = mse(y, predicted, 57)
        r2 = r2_score(y, predicted)

        #Mirem quina és la mitja del Player Efficiency Rating
        media = 0
        for prediction in predicted:
            media += prediction
        media /= len(predicted)

        print(i, " amb ", grau)
        print(media)
        print("Mean squeared error: ", MSE)
        print("R2 score: ", r2)
        print("----------------------------------------------------")
        dic[i, " grau: ", grau] = (media, MSE, r2, i, grau)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][2] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][2]
    if dic[clave][1] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][1]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][2], " i valor PER de ", dic[clave_r2][0])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][1], " i valor PER de ", dic[clave_mse][0])

CrossValidation lineal (Kobe)

In [ ]:
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
dic = {}
for names_cor in names_corr_importants:
    Dk_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(Dk_copy['PER'])
    for i in Dk_copy:
        if i not in names_cor:
            Dk_copy = Dk_copy.drop(i, axis = 1)

    x = Dk_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation Lasso (Kobe)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dk_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in Dk_copy:
        if i not in names_cor:
            Dk_copy = Dk_copy.drop(i, axis = 1)

    x = Dk_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation Bayes (Kobe)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)
dic = {}
for names_cor in names_combinations:
    Dk_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in Dk_copy:
        if i not in names_cor:
            Dk_copy = Dk_copy.drop(i, axis = 1)

    x = Dk_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation multilineal (Kobe)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

for names_cor in names_combinations:
    Dk_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in Dk_copy:
        if i not in names_cor:
            Dk_copy = Dk_copy.drop(i, axis = 1)

    x = Dk_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    media = 0
    for pred in predicted:
        media += pred
    media /= len(predicted)
    # Mostrem l'error (MSE i R2)
    print(media)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

CrossValidation ElasNet (Kobe)

In [ ]:
warnings.filterwarnings("ignore")
def split_data(x, y, train_ratio=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val


names_corr_importants = ["TS%", "USG%", "OBPM", "BPM", "OWS", "VORP"]
names_combinations = []
for i in range(2,6):
    names_combinations+=combinations(names_corr_importants,i)

for names_cor in names_combinations:
    Dk_copy = data_Kobe_no_estandaritzat.copy()
    y = np.array(data_Kobe_no_estandaritzat['PER'])
    for i in Dk_copy:
        if i not in names_cor:
            Dk_copy = Dk_copy.drop(i, axis = 1)

    x = Dk_copy.to_numpy()
        
    x_t, y_t, x_v, y_v = split_data(X, y)

    #regr = regression(X, y) 
    #regr = lasso(x_t, y_t, a = 0.001) 
    #regr = Bayes(X, y, t = 1e-12) 
    regr = ElasNet(X,y)
    predicted = regr.predict(x_t)

    # Mostrem la predicció del model entrenat en color vermell a la Figura anterior 1
    fig, ax = plt.subplots(figsize=(6, 3.84))

    aux = []
    for i,j in zip(x_t,predicted):
        aux.append(np.array([i[0],j]))
    aux = np.array(aux)
    aux = aux[aux[:,0].argsort()]

    xi = []
    yi = []

    for i in aux:
        xi.append(np.array([i[0]]))

    pred = regr.predict(xi)

    xi = []
    for i in aux:
        xi.append(i[0])
        yi.append(i[1])

    stdev = np.sqrt(sum((pred - y_t)**2) / (len(y_t) - 2))
    min_pred = pred - 0.675*stdev
    max_pred = pred + 0.675*stdev

    ax.scatter(X, np.array(y), marker='o', color = "gray")
    ax.plot(xi, np.array(pred), linestyle='-', label="OLS")
    ax.plot(xi, np.array(min_pred), linestyle='--', color='red', label="95% CI")
    ax.plot(xi, np.array(max_pred), linestyle='--', color='red')
    ax.fill_between(xi, np.array(min_pred), np.array(max_pred), alpha=0.1)

    # Mostrem l'error (MSE i R2)
    MSE = mse(list(y_v), list(regr.predict(x_v)),0)
    r2 = r2_score(y_v, regr.predict(x_v))

    print(names_cor)
    print("Mean squeared error: ", MSE)
    print("R2 score: ", r2)
    print("----------------------------------------------------")
    dic[names_cor] = (MSE, r2, names_cor)
    #print(names_cor)
    #print(media)
    #print("Mean squeared error: ", error)
    #print("R2 score: ", r2)
    
maxr2 = 0
minerror = 100
for clave in dic.keys():
    if dic[clave][1] > maxr2: #mira la r2 del diccionari
        clave_r2 = clave
        maxr2 = dic[clave][1]
    if dic[clave][0] < minerror: #mira el mse del diccionari
        clave_mse = clave
        minerror = dic[clave][0]


print("La combinació amb major r2 és ", clave_r2, " amb un valor de", dic[clave_r2][1])
print("La combinació amb menor mse és ", clave_mse, " amb un valor de", dic[clave_mse][0])

Cerca hiperparàmetres Lasso

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

dataset_advanced = load_dataset('advanced_stats.csv')
utils = []
for i in dataset_advanced.columns.values:
    if dataset_advanced[i].dtype != "object":
        utils.append(i)


for i in utils:
    x = np.array(dataset_advanced[i])
    X = x.reshape(x.shape[0], 1) 
y = np.array(dataset_advanced['PER'])
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = Lasso()
parameters = {'alpha': uniform(loc=0, scale=10),
              'tol'    : uniform(loc=10**(-12), scale=10**(-1)),
              'selection' : ["cyclic", "random"],
             }
randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                           cv = 2, n_iter = 10, n_jobs=-1)
randm_src.fit(X_train, y_train)

print(" Results from Random Search " )
print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)

Cerca hiperparàmetres BayessianRidge

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge

dataset_advanced = load_dataset('advanced_stats.csv')
utils = []
for i in dataset_advanced.columns.values:
    if dataset_advanced[i].dtype != "object":
        utils.append(i)

for i in utils:
    x = np.array(dataset_advanced[i])
    X = x.reshape(x.shape[0], 1) 
y = np.array(dataset_advanced['PER'])
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = BayesianRidge()
iters = []
for i in range(0,500,25):
    iters.append(i)
parameters = {'n_iter': iters,
              'tol'    : uniform(loc=10**(-12), scale=10**(-1)),
              'alpha_1': uniform(loc=10**(-12), scale=10**(-1)),
              'alpha_2': uniform(loc=10**(-12), scale=10**(-1)),
              'lambda_1': uniform(loc=10**(-12), scale=10**(-1)),
              'lambda_2': uniform(loc=10**(-12), scale=10**(-1))
             }
randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                           cv = 2, n_iter = 10, n_jobs=-1)
randm_src.fit(X_train, y_train)

print(" Results from Random Search " )
print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)